# Check for significance in HEIGHT subsystems

Integrating mouse KO data (from MGI)

8/19/21: updated to be consistent with BMI workflow

In [1]:
# load required packages

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import random

from IPython.display import display

import getpass
import ndex2

import json
import cdapsutil

# need ddot to parse the ontology
import ddot
from ddot import Ontology

# annotate the clusters
# gprofiler prelim annotation
from gprofiler import GProfiler
gp = GProfiler("MyToolName/0.1")

from statsmodels.stats import contingency_tables

import requests

# find human orthologs of mouse genes
import mygene
mg = mygene.MyGeneInfo()

from scipy.stats import hypergeom
from scipy.stats import norm

# latex rendering of text in graphs
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

sns.set(font_scale=1.4)

sns.set_style('white')

sns.set_style("ticks", {"xtick.major.size": 15, "ytick.major.size": 15})
plt.rcParams['svg.fonttype'] = 'none'

import sys

# % matplotlib inline

DISCLAIMIER: cdapsutil is experimental and may contain errors and interfaces may change


# Load interactome

In [2]:

interactome_uuid='4de852d9-9908-11e9-bcaf-0ac135e8bacf' # for PCNet
# interactome_uuid='275bd84e-3d18-11e8-a935-0ac135e8bacf' # for STRING high confidence
ndex_server='public.ndexbio.org'
ndex_user=None
ndex_password=None
G_int = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid=interactome_uuid
        ).to_networkx()
nodes = list(G_int.nodes)

# pcnet appears to have some self edges... should remove them. 
G_int.remove_edges_from(nx.selfloop_edges(G_int))

# print out interactome num nodes and edges for diagnostic purposes
print('number of nodes:')
print(len(G_int.nodes))
print('\nnumber of edges:')
print(len(G_int.edges))

number of nodes:
18820

number of edges:
2693109


In [3]:
nodes = list(G_int.nodes)

# First load HEIGHT overlap network computed in significance_network_overlap_sampling_210519.ipynb

In [4]:

ndex_server='public.ndexbio.org'
ndex_user=None
ndex_password=None
G_overlap_cx = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid='3c980793-0115-11ec-b666-0ac135e8bacf'
        )


# print out interactome num nodes and edges for diagnostic purposes
G_overlap = G_overlap_cx.to_networkx()
print('number of nodes:')
print(len(G_overlap.nodes))
print('\nnumber of edges:')
print(len(G_overlap.edges))

number of nodes:
488

number of edges:
7272


In [5]:
node_df = pd.DataFrame.from_dict(dict(G_overlap.nodes(data=True)), orient='index')
node_df.head()

,hHeight_seeds,z_rBL,z_hHeight,z_both,rBL_seeds,selected
REXO1L1P,0,2.03807984418225,2.073966593669306,4.226909512064731,0,NaN
SPINK9,0,3.4390189136039973,1.2535453450893113,4.310966150822391,0,NaN
ZNF781,1,2.0085379338411102,2.0659770974356313,4.149593370646417,0,NaN
GTSF1,0,2.638832019000988,1.571261150086052,4.1462942330593915,0,NaN
TPH1,1,3.8591196050315157,1.3833640822010471,5.33856745051849,0,NaN


In [6]:
# compute cosine similarity transformation
from netcoloc import network_colocalization

In [7]:
G_overlap_cosSim = network_colocalization.transform_edges(G_overlap,edge_weight_threshold=0.98)
print(len(G_overlap_cosSim.nodes()))
print(len(G_overlap_cosSim.edges()))

computing the adjacency matrix...
computing the cosine similarity...


/Users/brinrosenthal/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


rank transforming...
number of transformed edges returned = 
1336
488
1336


/Users/brinrosenthal/anaconda3/lib/python3.7/site-packages/netcoloc/network_colocalization.py:241: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sim_rank.values[[np.arange(sim_rank.shape[0])]*2] = 0


In [11]:
print(len(G_overlap_cosSim.nodes()))
print(len(G_overlap_cosSim.edges()))

488
1336


In [10]:
#upload G_cosSim to ndex
print(len(G_overlap_cosSim.nodes()))
print(len(G_overlap_cosSim.edges()))
G_overlap_cosSim_cx = ndex2.create_nice_cx_from_networkx(G_overlap_cosSim)
G_overlap_cosSim_cx.set_name('rat_human_HEIGHT_CosSim98') 
for node_id, node in G_overlap_cosSim_cx.get_nodes():
    data = node_df.loc[node['n']]
    for row, value in data.items():
        if row == 'd1_seeds' or row == 'd2_seeds' or row=='sum_seeds':
            data_type = 'double'
        elif row=='d1_name' or row=='d2_name':
            data_type='string'
        else:
            data_type = 'double'
        G_overlap_cosSim_cx.set_node_attribute(node_id, row, value, type=data_type)
        
        

# apply a template style (834b6ad4-d2ea-11eb-b666-0ac135e8bacf)
G_overlap_cosSim_cx.apply_template('ndexbio.org','2cbed84b-e5c3-11eb-b666-0ac135e8bacf')

SERVER = input('NDEx server (probably ndexbio.org): ')
USERNAME = input('NDEx user name: ')
PASSWORD = getpass.getpass('NDEx password: ')
network_uuid_NetColoc_CosSim = G_overlap_cosSim_cx.upload_to(SERVER, USERNAME, PASSWORD)

488
1336
NDEx server (probably ndexbio.org): ndexbio.org
NDEx user name: sbrosenthal@health.ucsd.edu
NDEx password: ········
Generating CX


# Build multiscale systems map



In [12]:
cd = cdapsutil.CommunityDetection()

# Run HiDeF on CDAPS REST service
G_hier = cd.run_community_detection(G_overlap_cx, algorithm='hidefv1.1beta',arguments={'--maxres':'10'})

In [13]:
# Print information about hierarchy
print('Hierarchy name: ' + str(G_hier.get_name()))
print('# nodes: ' + str(len(G_hier.get_nodes())))
print('# edges: ' + str(len(G_hier.get_edges())))

Hierarchy name: hidefv1.1beta_(none)_rat_human_height_sampling_z3_edges.txt
# nodes: 54
# edges: 54


In [14]:
G_hier = G_hier.to_networkx(mode='default')
G_hier

nodes = G_hier.nodes()

# print out interactome num nodes and edges for diagnostic purposes
print('number of nodes:')
print(len(G_hier.nodes()))
print('\nnumber of edges:')
print(len(G_hier.edges()))

number of nodes:
54

number of edges:
54


In [15]:
# add node attributes to dataframe for easier access
hier_df = pd.DataFrame.from_dict(dict(G_hier.nodes(data=True)), orient='index')
# relabel nodes in G_ASD
# G_ASD=nx.relabel_nodes(G_ASD,dict(hier_df['CD_CommunityName']))
hier_df['system_ID']=hier_df.index.tolist()
# some columns are not the right type
hier_df['CD_MemberList_Size']=[int(x) for x in hier_df['CD_MemberList_Size'].tolist()]
hier_df['HiDeF_persistence']=[int(x) for x in hier_df['HiDeF_persistence'].tolist()]
hier_df.head()

,CD_MemberList,CD_MemberList_Size,CD_Labeled,CD_MemberList_LogSize,CD_CommunityName,CD_AnnotatedMembers,CD_AnnotatedMembers_Size,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,HiDeF_persistence,represents,name,system_ID
0,CDC42EP3 PIH1D1 DRAP1 KDM8 PRR11 PLEKHG4B R3HD...,472,False,8.883,,,0,0.0,0.0,116,C1121,C1121,0
1,HDAC5 PRTFDC1 LOC102724428 YPEL3 MYEOV PBXIP1,6,False,2.585,,,0,0.0,0.0,13,C1152,C1152,1
2,SNAPC2 POLR2I HOXC4 VPS4A MTX1 CCDC85B,6,False,2.585,,,0,0.0,0.0,8,C1153,C1153,2
3,SMARCD2 MRPL49 VPS50 VPS51 DPF2,5,False,2.322,,,0,0.0,0.0,5,C1157,C1157,3
4,PBLD MED16 CRAT TP53INP2 CCDC184,5,False,2.322,,,0,0.0,0.0,5,C1158,C1158,4


# add fraction rat/human seeds

In [17]:
node_df.head()

,hHeight_seeds,z_rBL,z_hHeight,z_both,rBL_seeds,selected
REXO1L1P,0,2.03807984418225,2.073966593669306,4.226909512064731,0,NaN
SPINK9,0,3.4390189136039973,1.2535453450893113,4.310966150822391,0,NaN
ZNF781,1,2.0085379338411102,2.0659770974356313,4.149593370646417,0,NaN
GTSF1,0,2.638832019000988,1.571261150086052,4.1462942330593915,0,NaN
TPH1,1,3.8591196050315157,1.3833640822010471,5.33856745051849,0,NaN


In [18]:
hier_df.index=hier_df['name']
hier_df.head()

H_seeds_overlap = node_df[node_df['hHeight_seeds']=='1'].index.tolist()
print(len(H_seeds_overlap))

R_seeds_overlap = node_df[node_df['rBL_seeds']=='1'].index.tolist()
print(len(R_seeds_overlap))

num_d1_seeds, num_d2_seeds = [],[]
frac_d1_seeds, frac_d2_seeds=[],[]
systems_keep = []
for c in hier_df.index.tolist():
    system_genes = hier_df['CD_MemberList'].loc[c].split(' ')
    num_H_temp = len(list(np.intersect1d(system_genes,H_seeds_overlap)))
    num_R_temp = len(list(np.intersect1d(system_genes,R_seeds_overlap)))
    if (num_H_temp+num_H_temp)>0: # keep the system if it has at least 1 seed genes
        systems_keep.append(c)
        num_d1_seeds.append(num_H_temp)
        num_d2_seeds.append(num_R_temp)
        
        frac_d1_seeds.append(num_H_temp/np.float(len(system_genes)))
        frac_d2_seeds.append(num_R_temp/np.float(len(system_genes)))

        
frac_no_seeds = np.subtract(1.0,np.add(frac_d1_seeds,frac_d2_seeds))

hier_df = hier_df.loc[systems_keep]
hier_df['num_d1_seeds']=num_d1_seeds
hier_df['num_d2_seeds']=num_d2_seeds
hier_df['frac_d1_seeds']=frac_d1_seeds
hier_df['frac_d2_seeds']=frac_d2_seeds
hier_df['frac_no_seeds']=frac_no_seeds
print(len(hier_df))

hier_df.head()
    

307
127
54


,CD_MemberList,CD_MemberList_Size,CD_Labeled,CD_MemberList_LogSize,CD_CommunityName,CD_AnnotatedMembers,CD_AnnotatedMembers_Size,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,HiDeF_persistence,represents,name,system_ID,num_d1_seeds,num_d2_seeds,frac_d1_seeds,frac_d2_seeds,frac_no_seeds
name,,,,,,,,,,,,,,,,,,
C1121,CDC42EP3 PIH1D1 DRAP1 KDM8 PRR11 PLEKHG4B R3HD...,472,False,8.883,,,0,0.0,0.0,116,C1121,C1121,0,302,127,0.639831,0.269068,0.091102
C1152,HDAC5 PRTFDC1 LOC102724428 YPEL3 MYEOV PBXIP1,6,False,2.585,,,0,0.0,0.0,13,C1152,C1152,1,3,1,0.500000,0.166667,0.333333
C1153,SNAPC2 POLR2I HOXC4 VPS4A MTX1 CCDC85B,6,False,2.585,,,0,0.0,0.0,8,C1153,C1153,2,2,2,0.333333,0.333333,0.333333
C1157,SMARCD2 MRPL49 VPS50 VPS51 DPF2,5,False,2.322,,,0,0.0,0.0,5,C1157,C1157,3,4,2,0.800000,0.400000,-0.200000
C1158,PBLD MED16 CRAT TP53INP2 CCDC184,5,False,2.322,,,0,0.0,0.0,5,C1158,C1158,4,2,1,0.400000,0.200000,0.400000


In [19]:
# prune G_hier--> only keep systems with at least one seed gene

nkeep=[]
for n in list(G_hier.nodes()):
    if G_hier.nodes(data=True)[n]['name'] in systems_keep:
        nkeep.append(n)
        

G_hier = nx.subgraph(G_hier, nkeep)
print(len(G_hier.nodes()))
print(len(G_hier.edges()))

54
54


# NOTE: we don't undertake MPO validation for height at the moment (just BMI)

# gprofiler annotation of clusters

In [24]:

# print out GO pathways

# hier_df = hier_df.sort_values('name')

# ....this needs to be much more elegant...

system_name_list = []
for p in hier_df.index.tolist():
    focal_genes=hier_df['CD_MemberList'].loc[p].split(' ')
    print(p)
    print(len(focal_genes))
    if len(focal_genes)>2:
        gp_temp = pd.DataFrame(gp.profile(focal_genes,significance_threshold_method='fdr',
                                               sources=['GO:BP'],no_evidences=False))
        if len(gp_temp)>0: # make sure data is not empty
            
            # make sure terms are specific, and overlap with at least 3 genes
            gp_temp = gp_temp[(gp_temp['term_size']<1000)]
            gp_temp = gp_temp[gp_temp['intersection_size']>=3]
            
            gp_temp = gp_temp[gp_temp['p_value']<1E-3] # set a stringent pvalue threshold
            
            # try sorting by precision + recall
            #gp_temp['prec_plus_recall']=gp_temp['precision']+gp_temp['recall']
            gp_temp = gp_temp.sort_values('p_value',ascending=True)
            
            gp_temp.index=gp_temp['name']
            for t in gp_temp.index.tolist():
                if t.lower().find('dopamin')>-1:
                    print(gp_temp.loc[t][['p_value','intersections']])
                    
            gp_cols_keep = ['description','name','p_value','precision','recall',
                            'query_size','effective_domain_size','intersection_size','native','source',
                           'term_size','intersections']

            
            if len(gp_temp)>1:
                system_name_list.append(gp_temp.head(1)['name'].tolist()[0])
                gp_temp[gp_cols_keep].to_csv('../tables/rat_human_BMI_hierarchy_GO/'+str(p)+'.csv',sep=',',
                          index=False)
            else:
                system_name_list.append(p)
        else:
            system_name_list.append(p)
            

        display(gp_temp.head())
        
    else:
        system_name_list.append(p)

C1121
472


,description,effective_domain_size,intersection_size,intersections,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size,evidences
name,,,,,,,,,,,,,,,,
Rho protein signal transduction,"""A series of molecular signals within the cell...",18123,16,"[CDC42EP3, ARHGAP35, STARD13, CDC42EP2, CDC42E...",Rho protein signal transduction,GO:0007266,0.000191,[GO:0007265],0.037037,query_1,432,0.117647,True,GO:BP,136,"[[IBA], [ISS, IEA], [IBA], [IBA], [IBA], [IBA,..."
regulation of cell morphogenesis,"""Any process that modulates the frequency, rat...",18123,24,"[CDC42EP3, ARHGAP35, ZRANB1, CDC42EP2, CDC42EP...",regulation of cell morphogenesis,GO:0022604,0.000499,"[GO:0000902, GO:0022603]",0.055556,query_1,432,0.076923,True,GO:BP,312,"[[IBA, IEA], [IEA], [IMP], [IDA, IBA, IEA], [I..."
regulation of lipoprotein lipase activity,"""Any process that modulates the activity of th...",18123,7,"[ANGPTL4, GPIHBP1, APOC2, PCSK6, FURIN, ANGPTL...",regulation of lipoprotein lipase activity,GO:0051004,0.000770,[GO:0060191],0.016204,query_1,432,0.291667,True,GO:BP,24,"[[IDA, IEA], [IDA, IMP, IBA, TAS], [IDA, IBA],..."
actin cytoskeleton organization,"""A process that is carried out at the cellular...",18123,39,"[CDC42EP3, ARHGAP35, STARD13, CDC42BPG, CAP1, ...",actin cytoskeleton organization,GO:0030036,0.000770,"[GO:0007010, GO:0030029]",0.090278,query_1,432,0.054775,True,GO:BP,712,"[[IBA], [ISS, IEA], [IBA], [ISS, IBA, IEA], [I..."
regulation of cell shape,"""Any process that modulates the surface config...",18123,16,"[CDC42EP3, ARHGAP35, CDC42EP2, CDC42EP1, PLXNB...",regulation of cell shape,GO:0008360,0.000770,"[GO:0022604, GO:0065008]",0.037037,query_1,432,0.101911,True,GO:BP,157,"[[IBA, IEA], [IEA], [IDA, IBA, IEA], [IDA, IBA..."


C1152
6


""


C1153
6


...

""


C1173
4


,description,effective_domain_size,intersection_size,intersections,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size,evidences
name,,,,,,,,,,,,,,,,


C1166
4


""


In [25]:
system_name_list

['Rho protein signal transduction',
 'C1152',
 'C1153',
 'C1157',
 'C1158',
 'C1160',
 'C1161',
 'C1163',
 'C1164',
 'C1165',
 'C1167',
 'C1172',
 'regulation of lipoprotein lipase activity',
 'C1123',
 'mitochondrion organization',
 'sphingolipid biosynthetic process',
 'regulation of small GTPase mediated signal transduction',
 'C1142',
 'C1145',
 'C1149',
 'C1150',
 'histone demethylation',
 'elastic fiber assembly',
 'regulation of lipoprotein lipase activity',
 'C1159',
 'regulation of cell morphogenesis',
 'C1139',
 'C1141',
 'C1144',
 'cell fate commitment',
 'cleavage furrow formation',
 'C1156',
 'C1162',
 'C1169',
 'C1170',
 'C1171',
 'C1147',
 'C1168',
 'C1174',
 'mitochondrion organization',
 'C1143',
 'histone lysine demethylation',
 'C1130',
 'C1131',
 'C1140',
 'regulation of lipoprotein lipase activity',
 'C1134',
 'sphingolipid biosynthetic process',
 'C1135',
 'C1148',
 'C1154',
 'C1155',
 'C1173',
 'C1166']

# Load annotated hierarchy to ndex

In [27]:
d1='rat'
d2='human'
# add the best gprofiler annotation
MP_full_results_df = pd.DataFrame()
MP_full_results_df['gprofiler_name']=pd.Series(system_name_list,index=hier_df.index.tolist())
# don't annotate the root node
MP_full_results_df['gprofiler_name'].loc[root_node]=d1+'-'+d2+'HEIGHT systems map'

# also add the frac_seeds/num_seeds data here
MP_full_results_df=MP_full_results_df.join(hier_df[['num_d1_seeds','num_d2_seeds','frac_d1_seeds','frac_d2_seeds','frac_no_seeds']],
                                          how='left')

MP_full_results_df.head()

,gprofiler_name,num_d1_seeds,num_d2_seeds,frac_d1_seeds,frac_d2_seeds,frac_no_seeds
C1121,rat-humanHEIGHT systems map,302,127,0.639831,0.269068,0.091102
C1152,C1152,3,1,0.500000,0.166667,0.333333
C1153,C1153,2,2,0.333333,0.333333,0.333333
C1157,C1157,4,2,0.800000,0.400000,-0.200000
C1158,C1158,2,1,0.400000,0.200000,0.400000


In [28]:
# Convert G_hier to nice cx network
node_id_to_node_name = nx.get_node_attributes(G_hier, 'name')
for node_id in list(G_hier.nodes):
    del G_hier.nodes[node_id]['name']

G_hier_cx = ndex2.create_nice_cx_from_networkx(G_hier)

for node_id, node in G_hier_cx.get_nodes():
    node['n'] = node_id_to_node_name[node_id]

In [29]:
G_hier_cx.set_name('rat_human_HEIGHT_systems_map') 
for node_id, node in G_hier_cx.get_nodes():
    data = MP_full_results_df.loc[node['r']]
    #print(data)
    for row, value in data.items():
        if (row.find('gene_ids')>-1) or (row=='gprofiler_name'):
            data_type = "string"
            value=str(value)
        else:
            data_type = "double"
            value = str(value) # nice cx can only accept strings as values...
            if value=='inf': # check if inf, set to -1 if so
                value='-1'
            
        #print(value)
        #print(type(value))
        G_hier_cx.set_node_attribute(node_id, row, value, type=data_type)
        
# some CDAPs properties were corrupted on networkx conversion. Fix them here
for node_id, node in G_hier_cx.get_nodes():
    for i in np.arange(len(G_hier_cx.nodeAttributes[node_id])):
        dict_temp = G_hier_cx.nodeAttributes[node_id][i]
        if dict_temp['n'] in ['CD_MemberList_Size','CD_MemberList_LogSize','HiDeF_persistence']:
            G_hier_cx.set_node_attribute(node_id, dict_temp['n'], dict_temp['v'], type='double',overwrite=True)
            
# apply a template style (36041bac-d2e3-11eb-b666-0ac135e8bacf)
G_hier_cx.apply_template('ndexbio.org','36041bac-d2e3-11eb-b666-0ac135e8bacf')

In [30]:
#Upload to NDEx

SERVER = input('NDEx server (probably ndexbio.org): ')
USERNAME = input('NDEx user name: ')
PASSWORD = getpass.getpass('NDEx password: ')
network_uuid = G_hier_cx.upload_to(SERVER, USERNAME, PASSWORD)

NDEx server (probably ndexbio.org): ndexbio.org
NDEx user name: sbrosenthal@health.ucsd.edu
NDEx password: ········
Generating CX
